In [15]:
import pandas as pd
import string 
df_reviews = pd.read_csv('IMDB_Dataset.csv')

df_reviews_teste = df_reviews.sample(20)

df_filtrado_positivo = df_reviews_teste[df_reviews_teste['sentiment']=='positive']
df_filtrado_negativo = df_reviews_teste[df_reviews_teste['sentiment']=='negative']


dict_palavras_pos={}
for review in df_filtrado_positivo['review']:
    review=review.replace('<br />', " ")
    translator= str.maketrans(' ',' ', string.punctuation)
    review = review.translate(translator)
    review=review.split()
    for palavra in review:
        palavra=palavra.lower()
        if palavra in dict_palavras_pos:
            dict_palavras_pos[palavra]+=1
        else:
            dict_palavras_pos[palavra]=1


dict_palavras_neg={}
for review in df_filtrado_negativo['review']:
    review=review.replace('<br />', " ")
    translator= str.maketrans(' ',' ', string.punctuation)
    review = review.translate(translator)
    review=review.split()
    for palavra in review:
        palavra=palavra.lower()
        if palavra in dict_palavras_neg:
            dict_palavras_neg[palavra]+=1
        else:
            dict_palavras_neg[palavra]=1



In [16]:
#calcular P(palavra|classe)

#positivo
quant_review_pos = len(df_filtrado_positivo)
dict_prob_dado_pos = {}

for palavra, quant in dict_palavras_pos.items():
    prob = quant/quant_review_pos
    dict_prob_dado_pos[palavra] = prob



#negativo
quant_review_neg = len(df_filtrado_negativo)
dict_prob_dado_neg = {}

for palavra, quant in dict_palavras_neg.items():
    prob = quant/quant_review_neg
    dict_prob_dado_neg[palavra] = prob

#calcular P(nao palavra|classe)

#positvo
dict_prob_n_palavra_dado_pos = {}
for palavra, prob in dict_prob_dado_pos.items():
    prob_n_palavra = 1 - prob
    dict_prob_n_palavra_dado_pos[palavra] = prob_n_palavra

#negativo
dict_prob_n_palavra_dado_neg = {}
for palavra, prob in dict_prob_dado_neg.items():
    prob_n_palavra = 1 - prob
    dict_prob_n_palavra_dado_neg[palavra] = prob_n_palavra

In [17]:
#calcular P(palavra)

palavras_pos=len(df_filtrado_positivo)
palavras_neg=len(df_filtrado_negativo)
total_palavras=palavras_pos+palavras_neg

probalidade_palavras={}

for palavra in dict_palavras_pos:
    palavra_valor = 0
    try:
        palavra_valor += dict_palavras_neg[palavra]
    except:
        pass
    palavra_valor += dict_palavras_pos[palavra]
    p_palavra=palavra_valor/total_palavras
    probalidade_palavras[palavra]=p_palavra


for palavra in dict_palavras_neg:
    if not(palavra in probalidade_palavras):
        palavra_valor = 0
        try:
            palavra_valor += dict_palavras_pos[palavra]
        except:
            pass
        palavra_valor += dict_palavras_neg[palavra]
        p_palavra=palavra_valor/total_palavras
        probalidade_palavras[palavra]+=p_palavra


print(probalidade_palavras)


KeyError: 'ten'